# KIC 8462852 Century Long Data Analysis

## Context

KIC 8462852 is an enigmatic star (Boyajian et al. 2015).


## Content

- Data from the DASCH project: APASS input catalog, Kepler input catalog and GSC 2.3 catalog.
- Sonneberg and Sternberg observatory data provided by Michael Hippke.
- Maria Mitchell observatory data provided by Michael Castelaz.


## References

1. Boyajian, et al. (2015). Planet Hunters X. KIC 8462852 - Where's the Flux? arXiv:1509.03622
2. Castelaz and Barker (2018). KIC 8462852: Maria Mitchell Observatory Photographic Photometry 1922 to 1991. arXiv:1803.01943
3. Hippke et al. (2017). Sonneberg plate photometry for Boyajian's Star in two passbands. arXiv:1609.09290


In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it

In [2]:
import numpy as np # main library for numeric calculations
import pandas as pd # main library for data analysis
import matplotlib.pyplot as plt # main library for data plotting
import seaborn as sns # Another library for data plotting with more functions
sns.set()

import scipy as scp # STEM software
from scipy import signal
import scipy.interpolate as interpolate

from glob import glob # check files
from IPython.display import display, Markdown

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import lightkurve as lk
import astropy

import sys, os

file_path = os.getcwd()
print(file_path)
parent_dir = "\\".join(file_path.split("\\")[:-1])
print(parent_dir)

sys.path.append(parent_dir)

from kepler import *

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

import warnings
warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter


def printf(*args, sep=" ", end="\n"):
    string = sep.join([str(i) for i in args])+end
    display(Markdown(string))


def evaluateAndProcessData(x, y):
    t, c, k = interpolate.splrep(x, y, s=0, k=4)
    xx = np.linspace(x.min(), x.max(), 100)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    
    plt.figure(figsize=(16,6))
    plt.plot(x, y, 'bo', label='Original points')
    plt.plot(x, y)
    plt.plot(xx, spline(xx), 'r', label='BSpline')
    plt.grid()
    plt.legend(loc='best')
    plt.show()


c:\users\prannaya\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


C:\Users\Prannaya\ThreeBody\notebooks
C:\Users\Prannaya\ThreeBody


In [3]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
!pip install ipympl
%matplotlib widget

# Testing matplotlib interactions with a simple plot
fig = plt.figure()
fig.canvas.header_visible = False # Hide the Figure name at the top of the figure
fig.canvas.toolbar_visible = True

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
id = 8462852
# Retrieving all the important details
lc = retrieveKeplerLightCurve(id)
lc

In [ ]:
plotKeplerLightCurve(lc)
plt.show()

In [ ]:
plotKeplerSAPLightCurve(lc)
plt.show()

In [ ]:
plotKeplerPDCSAPLightCurve(lc)
plt.show()

In [ ]:
time = np.array(list(map(lambda time: time.value, list(lc.time))))
time

In [ ]:
flux = np.array(list(map(lambda flux: flux.value, list(lc.flux))))
flux

In [ ]:
evaluateAndProcessData(time, flux)